In [1]:
from pypots.imputation import SAITS
import pandas as pd
import numpy as np

2025-02-05 16:50:33 [ERROR]: No module named 'torch_geometric'
torch_geometric is missing, please install it with 'pip install torch_geometric' or 'conda install -c pyg pyg'
2025-02-05 16:50:33 [ERROR]: name 'MessagePassing' is not defined
Note torch_geometric is missing, please install it with 'pip install torch_geometric' or 'conda install -c pyg pyg'


In [2]:
def form_train(array, seq_len, step):
    result_list = []
    for i in range(seq_len, len(array) + 1, step):
        result_list.append(np.expand_dims(array[i - seq_len : i], axis=0))
    return np.concatenate(result_list, axis=0)

In [3]:
data = pd.read_csv("data.csv")
seq_len = 40
train_values = form_train(data.drop("Date", axis=1).values, seq_len, 1)
assert np.allclose(
    train_values[1][0][~np.isnan(train_values[1][0])], 
    train_values[0][1][~np.isnan(train_values[0][1])]
)

train_data = {"X": train_values}

In [5]:
model_params = {
    "n_layers": 2,
    "d_model": 64,
    "d_inner": 32,
    "n_heads": 4,
    "d_k": 32,
    "d_v": 32,
    "dropout": 0.1,
    "epochs": 20,
    "n_steps": seq_len,
    "n_features": train_data["X"].shape[-1],
    "epochs": 20
}
model = SAITS(**model_params)
# model.fit(train_data)
# model.save_model("weights", "pretrained_weights.pypots")
model.load_model("weights/pretrained_weights.pypots")

2025-02-05 16:50:43 [INFO]: No given device, using default device: cpu
2025-02-05 16:50:43 [WARNING]: saving_path not given. Model files and tensorboard file will not be saved.
2025-02-05 16:50:43 [INFO]: Model initialized successfully with the number of trainable parameters: 583,062
2025-02-05 16:50:43 [INFO]: Model loaded successfully from weights/pretrained_weights.pypots.


In [9]:
result = model.predict(train_data, return_latent_vars=True)